In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.polynomial.polynomial as poly
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
from scipy import fftpack

In [2]:
path= '/home/hcapettini/Escritorio/HDD1TB/Tesis/Analísis/Mayo/'             
#path = '/media/adminpc/Hilario-2/Tesis/Analísis/Mayo/'

simulacion='pulso_4' 
#simulacion='desequilibrio_5' 

pathr=path+simulacion+'/'+simulacion
pathw=path

#Tiempo a analizar
tmin_FT =0
tmax_FT =500
varstr='dens'
f_m = 0.5

In [3]:
radio= 3.e8;
largo= 25.e8;

c=2;                #ancho EN celdas por ejemplo cada 5 celdas Xcinco celdas hay un punto el ancho es 5
dc=0.1e8;           #Este es el ancho de cada celda de acuerdo al refinamiento
space = c * dc;
x0=space/2.;         #Indicar el punto inical el x
y0=5.e8 +space/2.; #Indicar el punto inical el y

nx=int(radio/(c*dc));
ny=int(largo/(c*dc));


In [4]:
for j in range (0,ny):  
    for i in range (0,nx):
        x=x0 +i*space
        y=y0 +j*space
        pos='_'+str(x/1.e8)+ '_'+str(y/1.e8)+ '_'
        with open(pathr +pos+ varstr+'.txt') as var:
            varlines = var.readlines()[:]
            time  = [float(line.split()[0]) for line in varlines]
            varx = [float(line.split()[1]) for line in varlines]
        time = np.array(time)
        variable = np.array(varx)
 
        nmin_FT = np.int(tmin_FT/f_m)
        nmax_FT = np.int(tmax_FT/f_m)
  
        variable_FT = variable[nmin_FT:nmax_FT]
        time_FT = time[nmin_FT:nmax_FT]
    
        p = np.poly1d(np.polyfit(time_FT, variable_FT, 2))
        t = np.linspace(tmin_FT, tmax_FT, np.size(time_FT))
        variable_FT = variable_FT-p(t)
        
        FT = fftpack.fft(variable_FT)
        freqs = fftpack.fftfreq(len(variable_FT), d=f_m)
        
        #Por alguna razón FT y freqs tiene todos los datos dos veces repetidos y por lo tanto 
        #si guardo la mitad de los datos tengo toda la información
        g=len(FT)/2 +1
        g=int(g)
        data = np.array([np.abs(freqs[0:g]), np.abs(FT[0:g])]) #Le tengo que poner un abs a la fcuencia porque sino me repite lo mismo en frecuencias negativas y en FT para eliminar el temrino complejo
        data = data.T
        datafile_path = pathw + simulacion + '/'+simulacion+pos+'TF_'+str(tmin_FT)+'_'+str(tmax_FT)+'_'+varstr+'.txt'
        with open(datafile_path, 'w+') as datafile_id:
            np.savetxt(datafile_id, data)
